# 資管四 唐瑋廷 B05705043 HW3

In [1]:
import numpy as np
import pandas as pd
from scipy.special import expit

## Q1.1

讀取&分析data

In [2]:
train_data = pd.read_csv('adult.data', skipinitialspace=True)
train_data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


把 binary variable 轉成 one-hot

In [3]:
train_x = pd.DataFrame(pd.get_dummies(train_data.drop('label', axis=1)))
train_x.shape

(32561, 108)

檢查 one-hot feature，剔除樣本數不到10的 feature

In [4]:
for c in train_x.columns[6:]: # first 6 columns are continuous varibles
    if sum(train_x[c]) < 10:
        train_x = train_x.drop(c, axis=1)
features = train_x.columns # use of test data
train_x.shape

(32561, 105)

labels

In [5]:
label_dict = {'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1}
train_y = train_data['label'].map(label_dict)

讀取 test data 並只保留 training data 有的 features

In [6]:
test_data = pd.read_csv('adult.test', skipinitialspace=True)
test_data.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,16281.000000,1.628100e+04,16281.000000,16281.000000,16281.000000,16281.000000
mean,38.767459,1.894357e+05,10.072907,1081.905104,87.899269,40.392236
std,13.849187,1.057149e+05,2.567545,7583.935968,403.105286,12.479332
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.167360e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.778310e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.383840e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000


In [7]:
test_x = pd.DataFrame(pd.get_dummies(test_data.drop('label', axis=1)))
test_x.shape

(16281, 107)

In [8]:
for c in test_x.columns[6:]: # first 6 columns are continuous varibles
    if c not in features:
        test_x = test_x.drop(c, axis=1)
test_x.shape

(16281, 105)

In [9]:
test_y = test_data['label'].map(label_dict)

In [10]:
train_x = train_x.to_numpy()
test_x = test_x.to_numpy()
train_y = train_y.to_numpy()
test_y = test_y.to_numpy()

## Q1.2

$$E(\omega) = \frac{1}{2}\omega^T\Lambda\omega - \sum_{i=1}^n\{t_nln(y_n) + (1-t_n)ln(1-y_n)\},\,y_n = \frac{1}{1+exp(-\omega^Tx_n)}$$\
$$\nabla E(\omega) = \frac{1}{2}(\Lambda^T + \Lambda)\omega + \sum_{i=1}^n (y_i - t_i)\phi_i = \Lambda\omega + \phi^T (y - t)$$\
$$H = \nabla\nabla E(\omega) = \Lambda + \sum_{i=1}^n y_i(1-y_i)\phi_i \phi_i^T = \Lambda + \phi R \phi^T$$
<center>where R is a NxN diagonal matrix with elements: $R_{nn} = y_n(1-y_n)$<\center>

## Q1.3

In [20]:
class mylogistic_l2():
    def __init__(self, reg_vec, max_iter = 1000, tol = 1e-5, add_intercept = True):
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
       
    def fit(self, x, t):
        # add intercept
        if self.add_intercept:
            x = np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)
    
        # init w
        b = self.reg_vec.trace() / self.reg_vec.shape[0]
        self.w = np.linalg.inv(x.T.dot(x) + b * np.identity(x.shape[1])).dot(x.T).dot(t)
        
        last_err = float("-inf")
        for _ in range(self.max_iter):
            # Newton-Raphson optimization method
            y = expit(self.w.T.dot(x.T))
            cur_err = self.err(y, t)
#             print(cur_err, (self.w**2).sum())
            if (cur_err - last_err) < self.tol:
                break
            last_err = cur_err
            
            gradient = self.reg_vec.dot(self.w) + x.T.dot(y-t)
            r = np.diagflat(y*(1-y))
            h = self.reg_vec + x.T.dot(r).dot(x)
            
            self.w = self.w - np.linalg.inv(h).dot(gradient)
            
            pred = np.where(y > 0.5, 1, 0)
            print(f'acc: {sum(np.equal(pred, t)) / t.shape[0]}')
    
    def predict(self, x):
        if self.add_intercept:
            x = np.concatenate((x, np.ones((x.shape[0], 1))), axis=1)
        p = expit(self.w.T.dot(x.T))
        return np.where(p > 0.5, 1, 0)
    
    def err(self, y, t):
        y = np.clip(y, 1e-12, 1.-1e-12)
        return np.sum(self.w.T.dot(self.reg_vec).dot(self.w))/2 - np.sum(t*np.log(y) + (1-t)*np.log(1-y)) / y.shape[0]

In [21]:
logic1 = mylogistic_l2(np.identity(train_x.shape[1]+1))

In [22]:
logic1.fit(train_x, train_y)

acc: 0.45087681582260986
acc: 0.8418353244679218
acc: 0.8503117226129419
acc: 0.8529222075489082
acc: 0.8533521697736556
acc: 0.8531986118362458
acc: 0.8531679002487639
acc: 0.8531679002487639


In [23]:
predict = logic1.predict(test_x)
sum(np.equal(predict, test_y)) / test_y.shape[0]

0.85283459246975

In [ ]:
expit(w.T.dot(test_x.T))

In [213]:
1 / (1 + np.exp(-w.T.dot(test_x.T)))

array([[0.48871118, 0.5582853 , 0.60106783, ..., 0.64225321, 0.56736559,
        0.67119986]])

## Q1.5

In [258]:
from sklearn.linear_model import LogisticRegression

In [335]:
clf = LogisticRegression(penalty='l2', max_iter=100)
clf.fit(train_x, train_y)
clf.predict(test_x)

array([0, 0, 0, ..., 0, 1, 0])

In [336]:
clf.score(test_x, test_y)

0.7978011178674529

In [281]:
tmp = test_x

In [294]:
w = np.linalg.inv(tmp.T.dot(tmp) + 1 * np.identity(tmp.shape[1])).dot(tmp.T).dot(test_y)

yy = expit(w.T.dot(tmp.T))
gradient = np.identity(tmp.shape[1]).dot(w) + tmp.T.dot(yy-test_y)
gradient.shape
r = np.diagflat(yy*(1-yy))
h = np.identity(tmp.shape[1]) + tmp.T.dot(r).dot(tmp)
new_w = w - np.linalg.inv(h).dot(gradient)
new_w.shape

(105,)

In [299]:
yy = np.clip(yy, 1e-12, 1.-1e-12)
np.sum(w.T.dot(np.identity(tmp.shape[1])).dot(w))/2 - np.sum(test_y*np.log(yy) + (1-test_y)*np.log(1-yy)) / yy.shape[0]

1.0665434373666685